In [1]:
import numpy as np
import tensorflow as tf
import data_utils as du

In [2]:
import random
from collections import defaultdict
import os

In [3]:
class Model:
    def __init__(self,configuration):
        tf.reset_default_graph()
        self.configuration = configuration
        self.X = tf.placeholder(shape=[None, self.configuration["contextSize"]-1],dtype=tf.int32)
        self.Y = tf.placeholder(shape=[None, self.configuration["contextSize"]-1],dtype=tf.int32)
        self.res_block = configuration["res_block"]
        self.train()

    def ConvolutionLayer(self,x, filter_shape):
        #with tf.variable_scope("layer_%d"%i):
        Wl = tf.Variable(tf.random_normal(filter_shape, stddev=1),name="linear_W")
        bl = tf.Variable(tf.constant(0.1, shape=[filter_shape[-1]]), name="linear_b")
        conv_w = tf.nn.bias_add(tf.nn.conv2d(x, Wl,strides=[1, 1, 1, 1],padding="SAME"),bl)
        
        Wv = tf.Variable(tf.truncated_normal(shape=filter_shape),name="gated_W")
        bv = tf.Variable(tf.constant(0.1, shape=[filter_shape[-1]]), name="gated_b")
        conv_v = tf.nn.bias_add(tf.nn.conv2d(x, Wv,strides=[1, 1, 1, 1],padding="SAME"),bv)
        
        h = conv_w * tf.nn.sigmoid(conv_v)
        
        return h
    
    def train(self):
        embeddingLayer = self.EmbeddingLayer()
        h = embeddingLayer
        res_input = h
        for i in xrange(self.configuration["numLayers"]):
            depth = h.get_shape()[-1]
            num_filter = self.configuration["filterSize"] if i<self.configuration["numLayers"]-1 else 1
            shape = [self.configuration["filterH"], self.configuration["embeddingSize"], int(depth), num_filter]
            with tf.variable_scope("layer_%d"%i):
                h = self.ConvolutionLayer(h, shape)
                if i%self.res_block == 0:
                    h += res_input
                    res_input = h

        h = tf.reshape(h, (-1, self.configuration["embeddingSize"]))
        print h.get_shape()
        y_shape = self.Y.get_shape().as_list()
        y_shape[0] = self.configuration["batchSize"]
        self.Y = tf.reshape(self.Y, (y_shape[0] * y_shape[1], 1))
        print self.Y.get_shape()
        softmax_w = tf.Variable(tf.random_normal([self.configuration["vocabSize"], self.configuration["embeddingSize"]], stddev=0.5), name="output_weights")
        print softmax_w.get_shape()
        softmax_b = tf.Variable(tf.random_normal([self.configuration["vocabSize"]], stddev=0.5), name="output_weights")
        self.Y = tf.cast(self.Y,tf.float32)
        self.loss = tf.reduce_mean(tf.nn.sampled_softmax_loss(softmax_w, softmax_b, self.Y,h, self.configuration["numSampled"], self.configuration["vocabSize"]))

        trainer = tf.train.MomentumOptimizer(self.configuration["learningRate"], self.configuration["momentum"])
        gradients = trainer.compute_gradients(self.loss)
        clipped_gradients = [(tf.clip_by_value(_[0], -self.configuration["gradClip"], self.configuration["gradClip"]), _[1]) for _ in gradients]
        self.optimizer = trainer.apply_gradients(clipped_gradients)
        self.perplexity = tf.exp(self.loss)
        print "Here"
        self.create_summaries()    
        
    def create_summaries(self):
        tf.summary.scalar("loss", self.loss)
        tf.summary.scalar("perplexity", self.perplexity)
        self.merged_summary_op = tf.summary.merge_all()
        
    def EmbeddingLayer(self):
        embeddings = tf.Variable(tf.random_normal([self.configuration["vocabSize"], self.configuration["embeddingSize"]], stddev=.01), name="embeddings")
        embed = tf.nn.embedding_lookup(embeddings, self.X)
        return tf.expand_dims(embed, -1)

In [4]:
# x,y, vocab_maping = get_data()

# vocab_size = len(vocab_maping)

In [5]:
# tf.nn.sampled_softmax_loss()
# tf.nn.nce_loss()

In [6]:
configuration = {"vocabSize":200,
                "embeddingSize":200,
                 "filterSize":64, 
                 "numLayers": 10, 
                 "blockSize":5, 
                 "filterH":5, 
                 "contextSize":20, 
                 "batchSize":20, 
                 "epochs":50, 
                 "numSampled":1, 
                 "learningRate": 0.1, 
                 "momentum": 0.99, 
                 "gradClip": 0.1, 
                 "numBatches": 0, 
                 "directory":"./Data/",
                 "filterW": 200,
                 "res_block": 2,
                 "summaryPath": "./"
                }

In [7]:
configuration["contextSize"] += configuration["filterH"]/2
x_batches, y_batches = du.prepare_data(configuration)
model = Model(configuration)

print "Started Model Training..."

batch_idx = 0
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    summary_writer = tf.summary.FileWriter(configuration["summaryPath"], graph=sess.graph)
    for i in xrange(configuration["epochs"]):
        for j in xrange(configuration["numBatches"]):
            inputs, labels, batch_idx = du.get_batch(x_batches, y_batches, batch_idx)
            print "AB Here"
            _, l = sess.run([model.optimizer, model.loss], feed_dict={model.X:inputs, model.Y:labels})
            print "Is k baad"
        print "Epoch: %.2f,  Loss: %.2f"%(i, l)

        if i % 10 == 0:
            perp = sess.run(model.perplexity, feed_dict={model.X:inputs, model.Y:labels})
            print "Perplexity: %.2f"%perp
        summaries = sess.run(model.merged_summary_op, feed_dict={model.X:inputs, model.Y:labels})
        summary_writer.add_summary(summaries, i)

(?, 200)
(420, 1)
(200, 200)
Here
Started Model Training...
AB Here
Is k baad
Epoch: 0.00,  Loss: 35810790251455053824.00


AttributeError: Model instance has no attribute 'y'